In [1]:
import urllib
import json
from pprint import pprint
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [2]:
# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:

url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "8c233bc2-1879-44ff-a0e4-9b69a9032c54"}
response = urllib.request.urlopen(url, data=bytes(json.dumps(params), encoding="utf-8"))
package = json.loads(response.read())
pprint(package)

{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=package_show',
 'result': {'creator_user_id': '150d5301-86ec-44a3-a070-50f2cea839c9',
            'dataset_category': 'Document',
            'excerpt': 'This dataset contains non-identifiable information '
                       'relating to each parking ticket issued for each '
                       'calendar year since 2008.',
            'formats': 'XLS,ZIP',
            'groups': [],
            'id': '8c233bc2-1879-44ff-a0e4-9b69a9032c54',
            'information_url': 'http://www.toronto.ca/pay-toronto-tickets/parkingtickets/index.htm',
            'is_retired': False,
            'isopen': True,
            'last_refreshed': '2019-07-23T17:51:57.153365',
            'license_id': 'other-open',
            'license_title': 'Other (Open)',
            'metadata_created': '2019-07-23T17:50:14.331462',
            'metadata_modified': '2019-11-06T15:23:09.564050',
            'name': 'parking-ti

In [3]:
filedata_2018_url = package["result"]["resources"][10]["url"]
filedata_2018_url

'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c233bc2-1879-44ff-a0e4-9b69a9032c54/resource/e0df3b42-8515-492b-ba85-a55c4cfd035f/download/parking-tickets-2018.zip'

In [4]:
# how to download the file from the above url
import requests
data_2018 = requests.get(filedata_2018_url)

In [5]:
#store the downloaded file in the parking2018.zip 
open('Resourses/parking2018.zip', 'wb').write(data_2018.content)

24403357

In [6]:
# extract all the files from the zip folder to current working directory
from zipfile import ZipFile

In [7]:
with ZipFile("Resourses/parking2018.zip","r") as zfile:
    zfile.extractall("Resourses")

In [8]:
data1 = pd.read_csv("Resourses/Parking_Tags_Data_2018_1.csv")
data2 = pd.read_csv("Resourses/Parking_Tags_Data_2018_2.csv")
data3 = pd.read_csv("Resourses/Parking_Tags_Data_2018_3.csv")

In [9]:
complete_data = pd.concat([data1,data2,data3])
complete_data.head()


,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***92517,20180101,16,PARK-WITHIN 9M INTERSECT ROAD,50,0.0,S/S,PRYOR AVE,E/O,CLOVERDALE RD,ON
1,***71708,20180101,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,266 DOVERCOURT RD,NaN,NaN,ON
2,***92311,20180101,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,15 FAIRBANK AVE,NaN,NaN,ON
3,***92312,20180101,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,15 FAIRBANK AVE,NaN,NaN,ON
4,***71709,20180101,29,PARK PROHIBITED TIME NO PERMIT,30,3.0,NR,266 DOVERCOURT RD,NaN,NaN,ON


In [10]:
len(complete_data)

2044288

In [11]:
complete_data.dtypes

tag_number_masked          object
date_of_infraction          int64
infraction_code             int64
infraction_description     object
set_fine_amount             int64
time_of_infraction        float64
location1                  object
location2                  object
location3                  object
location4                  object
province                   object
dtype: object

In [23]:
#toronto City Center -weather station data
#wData11 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCityCenter_2016.csv")
#wData12 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCityCenter_2017.csv")
#wData13 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCityCenter_2018.csv")
#Toronto City weather station Data
#wData21 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCity_2018.csv")
#wData22 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCity_2018.csv")
#wData23 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCity_2018.csv")
#Toronto Internantional Airport Data
wDataTIA2016 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoIntA_2016.csv")
wDataTIA2017 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoIntA_2017.csv")
wDataTIA2018 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoIntA_2018.csv")

In [26]:
#wDataTCC = pd.concat([wData11, wData12, wData13])
#wDataTC = pd.concat([wData21, wData22, wData23])
#weatherData = pd.merge(wDataTCC, wDataTC, on =["Date/Time", "Date/Time"] )
#df= weatherData[['Date/Time', 'Mean Temp (°C)_x','Mean Temp (°C)_y', 'Total Rain (mm)_x','Total Rain (mm)_y', 'Snow on Grnd (cm)_x', 'Total Snow (cm)_y' ]]
wDataTIA = pd.concat([wDataTIA2016, wDataTIA2017, wDataTIA2018])
combine_weather = wDataTIA[['Date/Time', 'Mean Temp (°C)','Total Rain (mm)','Total Snow (cm)']]
combine_weather.head()

,Date/Time,Mean Temp (°C),Total Rain (mm),Total Snow (cm)
0,2016-01-01,-2.3,0.0,0.8
1,2016-01-02,-2.0,0.0,0.0
2,2016-01-03,-5.0,0.0,0.0
3,2016-01-04,-13.3,0.0,0.0
4,2016-01-05,-8.9,0.0,0.0


In [27]:
combine_weather.rename(columns={"Date/Time" : "Date", "Total Rain (mm)" :"TotalRain(mm)", "Total Snow (cm)": "TotalSnow(cm)", "Mean Temp (°C)_y" : "Mean Temp_in_Celsius"}, inplace = True)

combine_weather.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Date,Mean Temp (°C),TotalRain(mm),TotalSnow(cm)
0,2016-01-01,-2.3,0.0,0.8
1,2016-01-02,-2.0,0.0,0.0
2,2016-01-03,-5.0,0.0,0.0
3,2016-01-04,-13.3,0.0,0.0
4,2016-01-05,-8.9,0.0,0.0


In [28]:
#df['DateTime'] = df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
complete_data['date_of_infraction'] = complete_data['date_of_infraction'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))


,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***92517,2018-01-01,16,PARK-WITHIN 9M INTERSECT ROAD,50,0.0,S/S,PRYOR AVE,E/O,CLOVERDALE RD,ON
1,***71708,2018-01-01,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,266 DOVERCOURT RD,NaN,NaN,ON
2,***92311,2018-01-01,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,15 FAIRBANK AVE,NaN,NaN,ON
3,***92312,2018-01-01,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,15 FAIRBANK AVE,NaN,NaN,ON
4,***71709,2018-01-01,29,PARK PROHIBITED TIME NO PERMIT,30,3.0,NR,266 DOVERCOURT RD,NaN,NaN,ON


In [37]:
complete_data.rename(columns={"date_of_infraction":"Date"})

,tag_number_masked,Date,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***92517,2018-01-01,16,PARK-WITHIN 9M INTERSECT ROAD,50,0.0,S/S,PRYOR AVE,E/O,CLOVERDALE RD,ON
1,***71708,2018-01-01,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,266 DOVERCOURT RD,NaN,NaN,ON
2,***92311,2018-01-01,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,15 FAIRBANK AVE,NaN,NaN,ON
3,***92312,2018-01-01,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,15 FAIRBANK AVE,NaN,NaN,ON
4,***71709,2018-01-01,29,PARK PROHIBITED TIME NO PERMIT,30,3.0,NR,266 DOVERCOURT RD,NaN,NaN,ON
5,***73023,2018-01-01,3,PARK ON PRIVATE PROPERTY,30,3.0,AT,735 COLLEGE ST,NaN,NaN,ON
6,***73024,2018-01-01,3,PARK ON PRIVATE PROPERTY,30,3.0,AT,735 COLLEGE ST,NaN,NaN,ON
7,***92313,2018-01-01,29,PARK PROHIBITED TIME NO PERMIT,30,3.0,NR,15 FAIRBANK AVE,NaN,NaN,ON
8,***71710,2018-01-01,29,PARK PROHIBITED TIME NO PERMIT,30,4.0,NR,262 DOVERCOURT RD,NaN,NaN,ON
9,***73025,2018-01-01,3,PARK ON PRIVATE PROPERTY,30,4.0,AT,735 COLLEGE ST,NaN,NaN,ON


In [39]:
complete_data.dtypes

tag_number_masked                 object
date_of_infraction        datetime64[ns]
infraction_code                    int64
infraction_description            object
set_fine_amount                    int64
time_of_infraction               float64
location1                         object
location2                         object
location3                         object
location4                         object
province                          object
dtype: object

In [38]:
final_df = complete_data.merge(combine_weather,on='Date', how ='left')

KeyError: 'Date'

In [48]:
final_df.head()

NameError: name 'final_df' is not defined